In [10]:
import pandas as pd
import my_data_frequency_conversor as dfc
#import my_models as mm
#import my_llm_ts_preprocessor as tsp
import my_prompting_tool as mpt 

In [11]:
# crear un diccionario con clave dataset y valor datos de training
datasets = {
    'ETH_HOURS':'DATA/ETHUSD-H1.csv',
    'ETH_DAYS':'DATA/ETHUSD-D1.csv',
    'ETH_MONTHS':'DATA/ETHUSD-Monthly.csv',
    
}
freqs = {
    'ETH_HOURS':'h',
    'ETH_DAYS':'d',
    'ETH_MONTHS':'MS'
}

descriptions = {
    'ETH_HOURS':mpt.paraphrase_initial('ETH_HOURS'),
    'ETH_DAYS':mpt.paraphrase_initial('ETH_DAYS'),
    'ETH_MONTHS':mpt.paraphrase_initial('ETH_MONTHS')
}
train_data = {}
val_data = {}
for ds in datasets:
    print(descriptions[ds])

This is an hourly time series dataset describing the price of Ethereum (ETH) in US dollars (USD), each value represents the average price of Ethereum in USD for that hour. 
This is a daily time series dataset describing the price of Ethereum (ETH) in US dollars (USD), each value represents the average price of Ethereum in USD for that day. 
This is a monthly time series dataset describing the price of Ethereum (ETH) in US dollars (USD), each value represents the average price of Ethereum in USD for that month. 


In [12]:
# Itera sobre cada conjunto de datos en el diccionario 'datasets'
for ds_name in datasets:  
    # Carga el archivo CSV correspondiente en un DataFrame de pandas
    df = pd.read_csv(datasets[ds_name])  
    
    # Divide los datos en entrenamiento y validación usando la función split_data de dfc
    train_data[ds_name], val_data[ds_name] = dfc.split_data(df)  
    
    # Ajusta la frecuencia de los datos de entrenamiento según el diccionario 'freqs'
    train_data[ds_name] = train_data[ds_name].asfreq(freqs[ds_name])  
    
    # Ajusta la frecuencia de los datos de validación según el diccionario 'freqs'
    val_data[ds_name] = val_data[ds_name].asfreq(freqs[ds_name])  
    
    # Redondea los valores de los datos de entrenamiento a 3 decimales
    train_data[ds_name] = train_data[ds_name].round(3)  
    
    # Redondea los valores de los datos de validación a 3 decimales
    val_data[ds_name] = val_data[ds_name].round(3)  
    
    # Imprime la forma (cantidad de filas y columnas) del conjunto de entrenamiento
    print(f'{ds_name} train data shape: {train_data[ds_name].shape}')  
    
    # Muestra las primeras filas del conjunto de entrenamiento
    print(train_data[ds_name].head())  
    
    # Imprime la forma del conjunto de validación
    print(f'{ds_name} val data shape: {val_data[ds_name].shape}')  
    
    # Muestra las primeras filas del conjunto de validación
    print(val_data[ds_name].head())  


ETH_HOURS train data shape: (26280, 1)
                           y
DateTime                    
2021-01-01 00:00:00  736.220
2021-01-01 01:00:00  733.981
2021-01-01 02:00:00  748.970
2021-01-01 03:00:00  744.460
2021-01-01 04:00:00  745.960
ETH_HOURS val data shape: (8784, 1)
                          y
DateTime                   
2024-01-01 00:00:00  2286.5
2024-01-01 01:00:00  2295.6
2024-01-01 02:00:00  2305.4
2024-01-01 03:00:00  2295.8
2024-01-01 04:00:00  2275.4
ETH_DAYS train data shape: (1095, 1)
                  y
DateTime           
2021-01-01   736.22
2021-01-02   729.46
2021-01-03   774.82
2021-01-04   976.32
2021-01-05  1041.20
ETH_DAYS val data shape: (366, 1)
                 y
DateTime          
2024-01-01  2286.5
2024-01-02  2352.7
2024-01-03  2358.2
2024-01-04  2211.3
2024-01-05  2271.0
ETH_MONTHS train data shape: (36, 1)
                   y
DateTime            
2021-01-01   736.220
2021-02-01  1313.869
2021-03-01  1424.400
2021-04-01  1919.300
2021-05-01  2769.40

In [13]:
for dataset in datasets:
    print("-" * 50)  # Línea divisoria
    print(f'Processing {dataset} dataset')
    train, test, seq = mpt.paraphrase_nlp(dataset, train_data[dataset], val_data[dataset])
    print()




--------------------------------------------------
Processing ETH_HOURS dataset
Longitud del entrenamiento: (26280, 1)
Longitud de la prueba: (8784, 1)
NaN detectado en: y    2137.31
Name: 2021-06-30 07:00:00, dtype: float64 y   NaN
Name: 2021-06-30 08:00:00, dtype: float64
NaN detectado en: y   NaN
Name: 2021-06-30 08:00:00, dtype: float64 y   NaN
Name: 2021-06-30 09:00:00, dtype: float64
NaN detectado en: y   NaN
Name: 2021-06-30 09:00:00, dtype: float64 y   NaN
Name: 2021-06-30 10:00:00, dtype: float64
NaN detectado en: y   NaN
Name: 2021-06-30 10:00:00, dtype: float64 y   NaN
Name: 2021-06-30 11:00:00, dtype: float64
NaN detectado en: y   NaN
Name: 2021-06-30 11:00:00, dtype: float64 y    2168.0
Name: 2021-06-30 12:00:00, dtype: float64
NaN detectado en: y    2168.0
Name: 2021-06-30 12:00:00, dtype: float64 y   NaN
Name: 2021-06-30 13:00:00, dtype: float64
NaN detectado en: y   NaN
Name: 2021-06-30 13:00:00, dtype: float64 y    2168.0
Name: 2021-06-30 14:00:00, dtype: float64
NaN d

In [14]:
# contar el número de tokens de args
import tiktoken as tk

encoding = tk.get_encoding("cl100k_base")  # Compatible con GPT-4 y GPT-3.5-turbo

print ("-" * 50)  # Línea divisoria
print("PROMPTS GENERALES EN LOS INPUTS:")
print ("-" * 20)  # Línea divisoria

# Contar tokens de prompt inicial
prompt_ini = "You are a helpful assistant that performs time series predictions. " \
             "The user will provide a sequence and you will predict the remaining sequence." \
             "The sequence is represented by decimal strings separated by commas. " \
             "Please continue the following sequence without producing any additional text. " \
             "Do not say anything like 'the next terms in the sequence are', just return the numbers."

num_tokens_prompt_ini = len(encoding.encode(prompt_ini))
print(f"Número de tokens del prompt inicial: {num_tokens_prompt_ini}")

ini_desc = "Predict the next 12 steps, where each step follows the format (from 1.0 increasing to 2.0) or (from 2.0 decreasing to 0.5)."\
                  " The final output should precisely follow the specified number of steps. Provide a sequence:\n"
num_tokens_ini_desc = len(encoding.encode(ini_desc))
print(f"Número de tokens del inicio de la descripción: {num_tokens_ini_desc}")

print ("-" * 50)  # Línea divisoria
print("PROMPTS ESPECÍFICOS DE CADA FRECUENCIA:")
print ("-" * 20)  # Línea divisoria

# Contar tokens de las secuencias:
for ds in datasets:
    print(f'Processing {ds} dataset')
    
    # Sustituyo nans, ya que solo quiero contar tokens
    new_train_data = train_data[ds].fillna(train_data[ds].mean())
    new_val_data = val_data[ds].fillna(val_data[ds].mean())

    # Secuencia de la serie temporal
    input_seq = mpt.paraphrase_seq2lan(new_train_data, descriptions[ds])
    num_tokens_input = len(encoding.encode(input_seq))
    print(f"Número de tokens solo de la serie temporal en input: {num_tokens_input}")

    # Input completo
    full_input = prompt_ini + ini_desc + input_seq
    num_tokens_full_input = len(encoding.encode(full_input))
    print(f"Número de tokens totales del input {ds}: {num_tokens_full_input}")

    # Output
    output_seq = mpt.paraphrase_seq2lan(new_val_data, '')
    num_tokens_output = len(encoding.encode(output_seq))
    print(f"Número de tokens del output {ds}: {num_tokens_output}")
    print ("-" * 50)  # Línea divisoria


--------------------------------------------------
PROMPTS GENERALES EN LOS INPUTS:
--------------------
Número de tokens del prompt inicial: 65
Número de tokens del inicio de la descripción: 56
--------------------------------------------------
PROMPTS ESPECÍFICOS DE CADA FRECUENCIA:
--------------------
Processing ETH_HOURS dataset
Número de tokens solo de la serie temporal en input: 368101
Número de tokens totales del input ETH_HOURS: 368222
Número de tokens del output ETH_HOURS: 123027
--------------------------------------------------
Processing ETH_DAYS dataset
Número de tokens solo de la serie temporal en input: 15347
Número de tokens totales del input ETH_DAYS: 15468
Número de tokens del output ETH_DAYS: 5111
--------------------------------------------------
Processing ETH_MONTHS dataset
Número de tokens solo de la serie temporal en input: 525
Número de tokens totales del input ETH_MONTHS: 646
Número de tokens del output ETH_MONTHS: 155
----------------------------------------